# NB06: Robustness Checks

**Goal**: Address two analytical gaps identified in the project review:
1. **H1b Formal Test** — statistically test whether stress-condition dark genes are more likely accessory than carbon/nitrogen dark genes
2. **Dark-vs-Annotated Concordance Control** — test whether dark gene cross-organism concordance patterns differ from annotated genes (H0 rejection)

**Requires**: BERDL JupyterHub (Spark access for specog query in Section 2)

**Inputs**: `data/dark_genes_integrated.tsv` (NB01), `data/concordance_scores.tsv` (NB02)

**Outputs**: `data/h1b_test_results.tsv`, `data/annotated_control_concordance.tsv`, `figures/fig16_h1b_test.png`, `figures/fig17_concordance_comparison.png`

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import fisher_exact, chi2_contingency, mannwhitneyu, ks_2samp
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Spark session
spark = get_spark_session()

# Project paths
if os.path.basename(os.getcwd()) == 'notebooks':
    PROJECT_DIR = os.path.dirname(os.getcwd())
else:
    PROJECT_DIR = os.getcwd()

DATA_DIR = os.path.join(PROJECT_DIR, 'data')
FIG_DIR = os.path.join(PROJECT_DIR, 'figures')
os.makedirs(FIG_DIR, exist_ok=True)

print(f'Project dir: {PROJECT_DIR}')

Project dir: /home/aparkin/BERIL-research-observatory/projects/functional_dark_matter


In [2]:
# Load NB01 unified table
unified = pd.read_csv(os.path.join(DATA_DIR, 'dark_genes_integrated.tsv'), sep='\t', low_memory=False)
unified['locusId'] = unified['locusId'].astype(str)
dark = unified[unified['is_dark'] == True].copy()
annotated = unified[unified['is_dark'] == False].copy()

# Load NB02 concordance scores for dark OGs
dark_concordance = pd.read_csv(os.path.join(DATA_DIR, 'concordance_scores.tsv'), sep='\t')

print(f'Unified table: {len(unified):,} genes')
print(f'  Dark: {len(dark):,}')
print(f'  Annotated: {len(annotated):,}')
print(f'Dark concordance OGs: {len(dark_concordance)}')

Unified table: 228,709 genes
  Dark: 57,011
  Annotated: 171,698
Dark concordance OGs: 65


## Section 1: H1b Formal Test

**Hypothesis H1b**: Dark genes important under stress conditions are more likely to be accessory (environment-specific) than dark genes important for carbon/nitrogen metabolism (which should be more core).

**Test**: Fisher's exact test on 2x2 contingency table: (stress vs carbon/nitrogen) × (core vs accessory)

In [3]:
# Filter to dark genes with both condition class and core/accessory info
h1b_data = dark[
    dark['top_condition_class'].notna() &
    (dark['is_core'].notna() | dark['is_auxiliary'].notna())
].copy()

# Classify conservation status
h1b_data['conservation'] = 'other'
h1b_data.loc[h1b_data['is_core'] == True, 'conservation'] = 'core'
h1b_data.loc[h1b_data['is_auxiliary'] == True, 'conservation'] = 'accessory'
h1b_data = h1b_data[h1b_data['conservation'].isin(['core', 'accessory'])]

# Classify condition groups
h1b_data['condition_group'] = 'other'
h1b_data.loc[h1b_data['top_condition_class'] == 'stress', 'condition_group'] = 'stress'
h1b_data.loc[h1b_data['top_condition_class'].isin(['carbon source', 'nitrogen source']), 'condition_group'] = 'carbon_nitrogen'
h1b_data = h1b_data[h1b_data['condition_group'].isin(['stress', 'carbon_nitrogen'])]

print(f'Dark genes testable for H1b: {len(h1b_data):,}')
print(f'\nCondition x Conservation crosstab:')
ct = pd.crosstab(h1b_data['condition_group'], h1b_data['conservation'])
print(ct)
print(f'\nWith row percentages:')
ct_pct = pd.crosstab(h1b_data['condition_group'], h1b_data['conservation'], normalize='index') * 100
print(ct_pct.round(1))

Dark genes testable for H1b: 7,491

Condition x Conservation crosstab:
conservation     accessory  core
condition_group                 
carbon_nitrogen        702  2050
stress                1088  3651

With row percentages:
conservation     accessory  core
condition_group                 
carbon_nitrogen       25.5  74.5
stress                23.0  77.0


In [4]:
# Build 2x2 contingency table
#                     Core    Accessory
# Stress              a       b
# Carbon/Nitrogen     c       d

a = int(ct.loc['stress', 'core'])
b = int(ct.loc['stress', 'accessory'])
c = int(ct.loc['carbon_nitrogen', 'core'])
d = int(ct.loc['carbon_nitrogen', 'accessory'])

table_2x2 = [[a, b], [c, d]]

# Fisher's exact test
odds_ratio_fisher, p_fisher = fisher_exact(table_2x2)

# Chi-squared test
chi2, p_chi2, dof, expected = chi2_contingency(table_2x2)

print('H1b Formal Test Results')
print('=' * 50)
print(f'\nContingency table:')
print(f'                    Core    Accessory    Total    % Accessory')
print(f'  Stress:          {a:>5d}      {b:>5d}     {a+b:>5d}       {100*b/(a+b):.1f}%')
print(f'  Carbon/Nitrogen: {c:>5d}      {d:>5d}     {c+d:>5d}       {100*d/(c+d):.1f}%')
print(f'\nH1b prediction: stress genes should have HIGHER % accessory')
print(f'Observed: stress = {100*b/(a+b):.1f}% accessory, carbon/nitrogen = {100*d/(c+d):.1f}% accessory')
if b/(a+b) > d/(c+d):
    print(f'Direction: CONSISTENT with H1b (stress more accessory)')
else:
    print(f'Direction: OPPOSITE to H1b (carbon/nitrogen more accessory)')
print(f'\nFisher\'s exact test:')
print(f'  Odds ratio: {odds_ratio_fisher:.3f}')
print(f'  p-value: {p_fisher:.4f}')
print(f'\nChi-squared test:')
print(f'  Chi2: {chi2:.2f}, dof: {dof}, p-value: {p_chi2:.4f}')
print(f'\nConclusion:', end=' ')
if p_fisher < 0.05:
    print(f'SIGNIFICANT at p < 0.05')
else:
    print(f'NOT significant at p < 0.05')

H1b Formal Test Results

Contingency table:
                    Core    Accessory    Total    % Accessory
  Stress:           3651       1088      4739       23.0%
  Carbon/Nitrogen:  2050        702      2752       25.5%

H1b prediction: stress genes should have HIGHER % accessory
Observed: stress = 23.0% accessory, carbon/nitrogen = 25.5% accessory
Direction: OPPOSITE to H1b (carbon/nitrogen more accessory)

Fisher's exact test:
  Odds ratio: 1.149
  p-value: 0.0134

Chi-squared test:
  Chi2: 6.09, dof: 1, p-value: 0.0136

Conclusion: SIGNIFICANT at p < 0.05


In [5]:
# Extended analysis: test all condition classes, not just stress vs carbon/nitrogen
h1b_all = dark[
    dark['top_condition_class'].notna() &
    (dark['is_core'] == True) | (dark['is_auxiliary'] == True)
].copy()
h1b_all['conservation'] = 'other'
h1b_all.loc[h1b_all['is_core'] == True, 'conservation'] = 'core'
h1b_all.loc[h1b_all['is_auxiliary'] == True, 'conservation'] = 'accessory'
h1b_all = h1b_all[h1b_all['conservation'].isin(['core', 'accessory'])]

# Per-condition accessory fraction
cond_conservation = h1b_all.groupby('top_condition_class').agg(
    n_total=('locusId', 'count'),
    n_core=('is_core', 'sum'),
    n_accessory=('is_auxiliary', 'sum'),
).reset_index()
cond_conservation['pct_accessory'] = 100 * cond_conservation['n_accessory'] / cond_conservation['n_total']
cond_conservation = cond_conservation.sort_values('pct_accessory', ascending=False)

print('Accessory fraction by condition class:')
print(cond_conservation.to_string(index=False))

Accessory fraction by condition class:
             top_condition_class  n_total n_core n_accessory pct_accessory
                       alp richb        1  False        True         100.0
                   sulfur source        1  False        True         100.0
                     rich moyls4        1  False        True         100.0
                metal limitation        6      1           5     83.333333
supernatant control:fungal media        6      1           5     83.333333
    r2a control with 0.2x vogels        3      1           2     66.666667
     varel_bryant_medium_glucose        5      2           3          60.0
                         control       50     22          28          56.0
                     r2a control       13      6           7     53.846154
                           mouse       43     20          23     53.488372
                              lb       82     45          37     45.121951
                           plant      101     57          44 

In [6]:
# Figure 16: H1b test visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel A: Grouped bar chart for stress vs carbon/nitrogen
ax = axes[0]
groups = ['Stress', 'Carbon/Nitrogen']
core_pcts = [100*a/(a+b), 100*c/(c+d)]
acc_pcts = [100*b/(a+b), 100*d/(c+d)]
x = np.arange(len(groups))
width = 0.35
bars1 = ax.bar(x - width/2, core_pcts, width, label='Core', color='#3498db', alpha=0.8)
bars2 = ax.bar(x + width/2, acc_pcts, width, label='Accessory', color='#e74c3c', alpha=0.8)
ax.set_ylabel('Percentage of dark genes')
ax.set_title(f'H1b Test: Conservation by Condition Class\n(Fisher p={p_fisher:.3f}, OR={odds_ratio_fisher:.2f})')
ax.set_xticks(x)
ax.set_xticklabels(groups)
ax.legend()
ax.set_ylim(0, 100)
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{bar.get_height():.1f}%', ha='center', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{bar.get_height():.1f}%', ha='center', fontsize=9)

# Panel B: All condition classes
ax = axes[1]
cc = cond_conservation[cond_conservation['n_total'] >= 10].copy()  # Only classes with N>=10
cc = cc.sort_values('pct_accessory')
y_pos = range(len(cc))
colors = ['#e74c3c' if c == 'stress' else '#3498db' if c in ('carbon source', 'nitrogen source') else '#95a5a6'
          for c in cc['top_condition_class']]
ax.barh(y_pos, cc['pct_accessory'].values, color=colors, alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels([f"{c} (n={n})" for c, n in zip(cc['top_condition_class'], cc['n_total'])], fontsize=9)
ax.set_xlabel('% Accessory')
ax.set_title('Accessory Fraction by Condition Class\n(red=stress, blue=carbon/nitrogen, gray=other)')
ax.axvline(x=cc['pct_accessory'].mean(), color='black', linestyle='--', alpha=0.5, label='Mean')
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'fig16_h1b_test.png'), dpi=150, bbox_inches='tight')
plt.close()
print('Saved fig16_h1b_test.png')

Saved fig16_h1b_test.png


In [7]:
# Save H1b test results
h1b_results = pd.DataFrame({
    'test': ['H1b_core_accessory'],
    'stress_core': [a],
    'stress_accessory': [b],
    'carbon_nitrogen_core': [c],
    'carbon_nitrogen_accessory': [d],
    'stress_pct_accessory': [100*b/(a+b)],
    'carbon_nitrogen_pct_accessory': [100*d/(c+d)],
    'fisher_odds_ratio': [odds_ratio_fisher],
    'fisher_p_value': [p_fisher],
    'chi2_statistic': [chi2],
    'chi2_p_value': [p_chi2],
    'direction_consistent_with_h1b': [b/(a+b) > d/(c+d)],
})
h1b_results.to_csv(os.path.join(DATA_DIR, 'h1b_test_results.tsv'), sep='\t', index=False)
print('Saved h1b_test_results.tsv')

Saved h1b_test_results.tsv


## Section 2: Dark-vs-Annotated Concordance Control

**Goal**: Test whether dark gene cross-organism concordance is a special property of dark genes, or whether annotated genes show similar patterns. If annotated genes also show high concordance (as expected — known functions should produce consistent phenotypes), then dark gene concordance is evidence that they behave like real functional genes, supporting H1.

**Approach**: Query `specog` for a matched sample of annotated-gene ortholog groups, compute concordance identically to NB02, and compare distributions.

In [8]:
# Identify annotated-gene OGs with 3+ FB organisms
# First, find OGs that contain ONLY annotated genes in FB
# (Using the specog ogId, which we need to look up via the specog table itself)

# Get the set of dark gene (orgId, locusId) pairs
dark_set = set(zip(dark['orgId'], dark['locusId'].astype(str)))

# Query specog to find all OGs and their members
# First get all unique ogIds and their member counts
all_specog_summary = spark.sql("""
    SELECT ogId,
           COUNT(DISTINCT orgId) as n_organisms,
           COUNT(DISTINCT CONCAT(orgId, ':', locusId)) as n_members
    FROM kescience_fitnessbrowser.specog
    GROUP BY ogId
    HAVING COUNT(DISTINCT orgId) >= 3
""").toPandas()

print(f'Total specog OGs with 3+ organisms: {len(all_specog_summary):,}')
print(f'Dark OG concordance scores: {len(dark_concordance)} OGs')

# The dark OGs are already known (from NB02)
dark_og_ids = set(dark_concordance['ogId'].astype(str))

# Candidate annotated OGs = all 3+ organism OGs minus dark OGs
annot_candidate_ogs = all_specog_summary[
    ~all_specog_summary['ogId'].astype(str).isin(dark_og_ids)
].copy()
print(f'Candidate annotated OGs (3+ organisms, not dark): {len(annot_candidate_ogs):,}')

Total specog OGs with 3+ organisms: 750
Dark OG concordance scores: 65 OGs
Candidate annotated OGs (3+ organisms, not dark): 685


In [9]:
# Stratified sampling: match the organism-count distribution of the 65 dark OGs
dark_org_dist = dark_concordance['total_organisms'].value_counts().sort_index()
print('Dark OG organism-count distribution:')
print(dark_org_dist)

# For each organism count bin, sample proportionally (scaled up to ~500 total)
np.random.seed(42)
scale_factor = min(500 / len(dark_concordance), len(annot_candidate_ogs) / len(dark_concordance))
scale_factor = min(scale_factor, 8)  # Cap at 8x

sampled_annot_ogs = []
for n_orgs, count in dark_org_dist.items():
    target_n = max(int(count * scale_factor), count)  # At least match dark count
    pool = annot_candidate_ogs[annot_candidate_ogs['n_organisms'] == n_orgs]
    if len(pool) == 0:
        # Try nearest bin
        pool = annot_candidate_ogs[
            (annot_candidate_ogs['n_organisms'] >= n_orgs - 1) &
            (annot_candidate_ogs['n_organisms'] <= n_orgs + 1)
        ]
    sample_n = min(target_n, len(pool))
    if sample_n > 0:
        sampled = pool.sample(n=sample_n, random_state=42)
        sampled_annot_ogs.append(sampled)

sampled_annot = pd.concat(sampled_annot_ogs, ignore_index=True)
print(f'\nSampled annotated OGs: {len(sampled_annot)}')
print(f'Organism-count distribution of sample:')
print(sampled_annot['n_organisms'].value_counts().sort_index())

Dark OG organism-count distribution:
total_organisms
3     35
4     16
5      3
6      3
7      3
8      3
10     2
Name: count, dtype: int64

Sampled annotated OGs: 490
Organism-count distribution of sample:
n_organisms
3     269
4     123
5      23
6      23
7      23
8      23
10      6
Name: count, dtype: int64


In [10]:
# Query specog for sampled annotated OGs (identical pattern to NB02 cell-11)
annot_og_ids = sampled_annot['ogId'].tolist()
og_spark = spark.createDataFrame([(str(og),) for og in annot_og_ids], ['ogId'])
og_spark.createOrReplaceTempView('target_annot_ogs')

annot_specog = spark.sql("""
    SELECT s.ogId, s.expGroup, s.condition, s.orgId, s.locusId,
           CAST(s.minFit AS FLOAT) as minFit,
           CAST(s.maxFit AS FLOAT) as maxFit,
           CAST(s.minT AS FLOAT) as minT,
           CAST(s.maxT AS FLOAT) as maxT,
           CAST(s.nInOG AS INT) as nInOG
    FROM kescience_fitnessbrowser.specog s
    JOIN target_annot_ogs t ON s.ogId = t.ogId
""").toPandas()

print(f'Annotated specog entries: {len(annot_specog):,}')
print(f'OGs represented: {annot_specog["ogId"].nunique()}')

Annotated specog entries: 1,962
OGs represented: 490


In [11]:
# Compute concordance for annotated OGs (identical method to NB02 cell-12/13)

# Strong effect thresholds (same as NB02)
annot_specog['has_strong_effect'] = (annot_specog['minFit'].abs() > 1) & (annot_specog['minT'].abs() > 3)

# Per OG x expGroup: count organisms with strong effects
annot_concordance_raw = annot_specog.groupby(['ogId', 'expGroup']).agg(
    n_organisms=('orgId', 'nunique'),
    n_strong=('has_strong_effect', 'sum'),
    min_of_minFit=('minFit', 'min'),
    mean_minFit=('minFit', 'mean'),
).reset_index()

annot_concordance_raw['concordance_frac'] = annot_concordance_raw['n_strong'] / annot_concordance_raw['n_organisms']

# Per-OG summary: max concordance across condition classes
annot_og_concordance = annot_concordance_raw.sort_values('concordance_frac', ascending=False).groupby('ogId').agg(
    best_condition=('expGroup', 'first'),
    max_concordance=('concordance_frac', 'max'),
    n_condition_classes=('expGroup', 'nunique'),
    total_organisms=('n_organisms', 'max'),
).reset_index()

print(f'Annotated OG concordance summary: {len(annot_og_concordance)} OGs')
print(f'\nAnnotated concordance distribution:')
print(annot_og_concordance['max_concordance'].describe())
print(f'\nDark concordance distribution (from NB02):')
print(dark_concordance['max_concordance'].describe())

Annotated OG concordance summary: 490 OGs

Annotated concordance distribution:
count    490.000000
mean       0.985420
std        0.084566
min        0.333333
25%        1.000000
50%        1.000000
75%        1.000000
max        1.333333
Name: max_concordance, dtype: float64

Dark concordance distribution (from NB02):
count    65.000000
mean      0.976007
std       0.089940
min       0.500000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: max_concordance, dtype: float64


In [12]:
# Statistical comparison: dark vs annotated concordance
dark_scores = dark_concordance['max_concordance'].values
annot_scores = annot_og_concordance['max_concordance'].values

# Mann-Whitney U test
stat_mw, p_mw = mannwhitneyu(dark_scores, annot_scores, alternative='two-sided')

# KS test
stat_ks, p_ks = ks_2samp(dark_scores, annot_scores)

print('Dark vs. Annotated Concordance Comparison')
print('=' * 50)
print(f'\nDark OGs:      n={len(dark_scores)}, median={np.median(dark_scores):.3f}, mean={np.mean(dark_scores):.3f}')
print(f'Annotated OGs: n={len(annot_scores)}, median={np.median(annot_scores):.3f}, mean={np.mean(annot_scores):.3f}')
print(f'\nMann-Whitney U test:')
print(f'  U statistic: {stat_mw:.1f}')
print(f'  p-value: {p_mw:.4e}')
print(f'\nKolmogorov-Smirnov test:')
print(f'  KS statistic: {stat_ks:.3f}')
print(f'  p-value: {p_ks:.4e}')
print(f'\nInterpretation:', end=' ')
if p_mw < 0.05:
    if np.median(dark_scores) > np.median(annot_scores):
        print('Dark genes show HIGHER concordance than annotated genes.')
        print('This is unexpected and may reflect selection bias (only OGs with 3+ organisms tested).')
    else:
        print('Annotated genes show HIGHER concordance than dark genes.')
        print('Dark gene concordance is real but weaker than annotated genes, consistent with partial/noisy function.')
else:
    print('No significant difference in concordance between dark and annotated genes.')
    print('Dark genes show comparable concordance to annotated genes — supporting H1 (they behave like real functional genes).')

Dark vs. Annotated Concordance Comparison

Dark OGs:      n=65, median=1.000, mean=0.976
Annotated OGs: n=490, median=1.000, mean=0.985

Mann-Whitney U test:
  U statistic: 15275.0
  p-value: 1.7242e-01

Kolmogorov-Smirnov test:
  KS statistic: 0.034
  p-value: 1.0000e+00

Interpretation: No significant difference in concordance between dark and annotated genes.
Dark genes show comparable concordance to annotated genes — supporting H1 (they behave like real functional genes).


In [13]:
# Figure 17: Concordance comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel A: Side-by-side histograms
ax = axes[0]
bins = np.linspace(0, 1, 21)
ax.hist(annot_scores, bins=bins, alpha=0.6, color='#3498db', label=f'Annotated (n={len(annot_scores)})', density=True)
ax.hist(dark_scores, bins=bins, alpha=0.6, color='#e74c3c', label=f'Dark (n={len(dark_scores)})', density=True)
ax.set_xlabel('Max concordance fraction')
ax.set_ylabel('Density')
ax.set_title(f'Cross-Organism Concordance: Dark vs Annotated\n(Mann-Whitney p={p_mw:.3e})')
ax.legend()

# Panel B: CDF comparison
ax = axes[1]
for scores, label, color in [
    (annot_scores, f'Annotated (n={len(annot_scores)})', '#3498db'),
    (dark_scores, f'Dark (n={len(dark_scores)})', '#e74c3c')
]:
    sorted_scores = np.sort(scores)
    cdf = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
    ax.plot(sorted_scores, cdf, label=label, color=color, linewidth=2)
ax.set_xlabel('Max concordance fraction')
ax.set_ylabel('Cumulative fraction of OGs')
ax.set_title(f'CDF Comparison (KS stat={stat_ks:.3f}, p={p_ks:.3e})')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'fig17_concordance_comparison.png'), dpi=150, bbox_inches='tight')
plt.close()
print('Saved fig17_concordance_comparison.png')

Saved fig17_concordance_comparison.png


In [14]:
# Save annotated concordance control data
annot_og_concordance.to_csv(os.path.join(DATA_DIR, 'annotated_control_concordance.tsv'), sep='\t', index=False)
print(f'Saved annotated_control_concordance.tsv ({len(annot_og_concordance)} OGs)')

Saved annotated_control_concordance.tsv (490 OGs)


## Section 3: Summary

In [15]:
print('=' * 70)
print('NB06 SUMMARY: ROBUSTNESS CHECKS')
print('=' * 70)

print(f'\n--- H1b Formal Test ---')
print(f'Dark genes tested: {len(h1b_data):,}')
print(f'Stress: {a+b} genes ({100*b/(a+b):.1f}% accessory)')
print(f'Carbon/Nitrogen: {c+d} genes ({100*d/(c+d):.1f}% accessory)')
print(f'Fisher\'s exact: OR={odds_ratio_fisher:.3f}, p={p_fisher:.4f}')
print(f'Direction consistent with H1b: {b/(a+b) > d/(c+d)}')

print(f'\n--- Dark vs Annotated Concordance ---')
print(f'Dark OGs: {len(dark_scores)} (median concordance: {np.median(dark_scores):.3f})')
print(f'Annotated OGs: {len(annot_scores)} (median concordance: {np.median(annot_scores):.3f})')
print(f'Mann-Whitney U: p={p_mw:.4e}')
print(f'KS test: D={stat_ks:.3f}, p={p_ks:.4e}')

print(f'\n--- Output Files ---')
for f in ['h1b_test_results.tsv', 'annotated_control_concordance.tsv']:
    fp = os.path.join(DATA_DIR, f)
    if os.path.exists(fp):
        print(f'  {f}: {os.path.getsize(fp)/1024:.1f} KB')
for f in ['fig16_h1b_test.png', 'fig17_concordance_comparison.png']:
    fp = os.path.join(FIG_DIR, f)
    if os.path.exists(fp):
        print(f'  {f}')

print('\n' + '=' * 70)

NB06 SUMMARY: ROBUSTNESS CHECKS

--- H1b Formal Test ---
Dark genes tested: 7,491
Stress: 4739 genes (23.0% accessory)
Carbon/Nitrogen: 2752 genes (25.5% accessory)
Fisher's exact: OR=1.149, p=0.0134
Direction consistent with H1b: False

--- Dark vs Annotated Concordance ---
Dark OGs: 65 (median concordance: 1.000)
Annotated OGs: 490 (median concordance: 1.000)
Mann-Whitney U: p=1.7242e-01
KS test: D=0.034, p=1.0000e+00

--- Output Files ---
  h1b_test_results.tsv: 0.4 KB
  annotated_control_concordance.tsv: 12.8 KB
  fig16_h1b_test.png
  fig17_concordance_comparison.png

